# Notes

This assignment is devoted to `pandas`. It covers indexing and filtering, and some `groupby` and `join` operations. The assignment roughly corresponds to Week 4 and the beginning of Week 5 of the course.

The main dataset you'll be using is [Titanic](https://www.kaggle.com/c/titanic). Please, note, that you must not rely on any specific location for the dataset, hence, any code like

```python
titanic_train = pd.read_csv("<location>/train.csv")
```

will fail and your notebook won't be validated and graded. Inputs to the functions are described explicitly in each case, and that's the only thing you can rely on.

In [1]:
%pylab inline
plt.style.use("bmh")

Populating the interactive namespace from numpy and matplotlib


In [2]:
plt.rcParams["figure.figsize"] = (6,6)

In [3]:
import numpy as np
import pandas as pd
import re

In [4]:
STUDENT = "Yehoshua Friedman"
ASSIGNMENT = 4
TEST = False

In [5]:
if TEST:
    import solutions
    total_grade = 0
    MAX_POINTS = 16

# Indexing and filtering

### 1. Fixing age (1 point).

There are several known mistakes in the Titanic dataset.

Namely, [Julia Florence Siegel](https://www.encyclopedia-titanica.org/titanic-survivor/julia-florence-cavendish.html) (Mrs. Tyrell William Cavendish) is mistakenly marked as being 76 years old (the age she actually died, but many years after Titanic).

You must **replace the corresponding age value in the dataframe with her actual age at the time** (25) and return the dataset. Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. You must return a copy of the input dataframe, and not perform replacement in the original dataframe. Structure and indexing must be the same as in input.

In [6]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

df = pd.concat([train, test]).reset_index().set_index('PassengerId')

In [7]:

df.loc[df['Name'] == "Cavendish, Mrs. Tyrell William (Julia Florence Siegel)", 'Age'] = 25

In [8]:
def fix_age(df):
    df.loc[df['Name'] == "Cavendish, Mrs. Tyrell William (Julia Florence Siegel)", 'Age'] = 25
    return df

In [9]:
PROBLEM_ID = 1

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

### 2. Embarkment port distribution (1 point).

You must find the value counts for embarkment port (`Embarked` column) for the passengers, who travelled in 3-d class, were male and between 20 and 30 years old (both inclusive). No need to treat missing values separately.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. You must return **series**, indexed with values from `Embarked`, according to `.value_counts()` method semantics:

```
S    <number of passengers in 3-d class, embarked at S, 20<=Age<=30>
C    <...>
Q    <...>
Name: Embarked, dtype: int64
```

In [10]:
df[(df['Pclass'] == 3) & (df['Age'] > 20) & (df['Age']<30)].Embarked.value_counts()

S    153
C     17
Q     13
Name: Embarked, dtype: int64

In [11]:
def embarked_stats(df):
    return df[(df['Pclass'] == 3) & (df['Age'] > 20) & (df['Age']<30)].Embarked.value_counts()

In [12]:
PROBLEM_ID = 2

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, embarked_stats)

### 3. Fill missing age values (1 point).

Some age values are missing in the Titanic dataset. You need to calculate average age over all passengers, and fill missing age values in `Age` column.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **new** dataframe with the same structure, but without missing values in `Age` column.

In [13]:
def fix_age(df):
    df = df['Age'].fillna(df['Age'].mean())
    return df

In [14]:
PROBLEM_ID = 3

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

### 4. Child travelling alone (1 point).

You must find a child (`Age<10`) on-board, who was travelling without siblings or parents and find a name of her nursemaid.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **tuple** of two strings, collected from `Name` column, with one being child's name and second being nursemaid's name. It's known, that there's **only one child** like this.

In [15]:
# child = df.loc[(df['Age']<10) & (df['SibSp'] == 0) & (df['Parch'] == 0), ['Name']].values[0][0]

# ticket = df.loc[df['Name']== child, 'Ticket'].values[0]
# ticket
# tuple(df.loc[df['Ticket'] == ticket, 'Name'].values)

ticket = df.loc[(df['Age']<10) & (df['SibSp'] == 0) & (df['Parch'] == 0), ['Ticket']].values[0][0]
tuple(df.loc[df['Ticket'] == ticket, 'Name'].values)


('Dowdell, Miss. Elizabeth', 'Emanuel, Miss. Virginia Ethel')

In [16]:
def get_nursemaid(df):
    ticket = df.loc[(df['Age']<10) & (df['SibSp'] == 0) & (df['Parch'] == 0), ['Ticket']].values[0][0]
    return tuple(df.loc[df['Ticket'] == ticket, 'Name'].values)

In [17]:
PROBLEM_ID = 4

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_nursemaid)

### 5. Port with the most children embarked (1 point).

You must find, which port had the largest percentage of children (`Age<10`) embarked, i.e. number of children divided by total number of passengers embarked.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **single string** with port letter.

In [18]:
def get_port(df):
    x = df.loc[df['Age'] <10].Embarked.value_counts()/ len(df)
    return x.idxmax()

In [19]:
PROBLEM_ID = 5

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_port)

### 6. Passengers per ticket (2 points).

Calculate average and maximum number of passengers per ticket.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **tuple** of two values - average and maximum number of passengers per ticket.

In [20]:
def get_ticket_stats(df):
    return (df['Ticket'].value_counts().mean(), df['Ticket'].value_counts().max())
# get_ticket_stats(df)

In [21]:
PROBLEM_ID = 6

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_ticket_stats)

### 7. Fare per passenger (3 points).

For each individual ticket, you must calculate **fare per person for that ticket**, and then calculate averages for each class. Note, that you will need to apply `groupby` and you may consider using `.first()` of resulting `DataFrameGroupBy`. Also, caferully consider, in which order calculations are performed.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `pd.Series` with three elements, indexed by class:

```
1    <average per person fare in class 1>
2    <...>
3    <...>
Name: Pclass, dtype: float64
```

In [22]:
def get_fare_per_pass(df):
    return (df.groupby(['Pclass','Ticket'])['Fare'].sum()/ df.groupby(['Pclass', 'Ticket'])['Ticket'].count()).unstack().mean(axis=1)

In [23]:
PROBLEM_ID = 7

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_fare_per_pass)

### 8. Fill missing age values (3 points).

In problem 3 you filled missing age values with global average over all passengers. Now, you need to fill them **according to class and sex**. For example, for a female passenger from 2d class, missing age value must be filled with average age of females in 2d class.

In this problem, you may need joins and `.apply()`, although there are several ways to get the same result.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **new** dataframe with the same structure as input, but without missing values in `Age` column.

In [24]:
df.groupby(['Pclass', 'Sex'])[['Age']].mean()


Age
Pclass Sex              
1      female  36.654135
       male    41.029272
2      female  27.499223
       male    30.815380
3      female  22.185329
       male    25.962264

In [25]:
# df.groupby(['Pclass', 'Sex'])['Age'].apply(lambda x: x.fillna(x.mean()))
# #x is a variable for the row in the groupby. We are saying to apply a function to all values which are grouped within each row in our groupby

In [26]:
def fix_age_groupped(df):
    return df.groupby(['Pclass', 'Sex'])['Age'].apply(lambda x: x.fillna(x.mean()))


In [27]:
PROBLEM_ID = 8

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age_groupped)

### 9. Finding couples (3 points).

Based on the code from Lecture 5, build a dataframe of couples. Filter it by survival status: select those couples, in which only one of spouses survived or none of two. Built survival statistics by class, i.e. ratio of the number couples with partial survival or couples which died together, divided by total number of couples in class. If the survival status of one or both of spouses is not known, it must be considered as `0`.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `Series` with three elements indexed by values from `Pclass` column (see P7 as a reference).

In [45]:
def find_couples(df):
    family_names = (df
                .replace(re.compile(r'\s+\(.*\)'), '') #remove all letters inside ( )
                .replace(re.compile("Mrs."), "Mr."))[["Name",'Pclass', "Sex", 'Survived']]
    family_names = family_names[(family_names['Sex'] == 'female') & family_names.Name.str.contains("Mr.")]
    couples = df.join(family_names.reset_index().set_index('Name')[['Pclass','PassengerId', 'Survived']]
        , on = 'Name', how = 'inner', rsuffix= '_spouse')   
    couples.rename({"PassengerId":"PassengerId_Spouse"},
                axis=1, inplace=True)
                
    survived = couples[(couples['Survived'] == 1) & (couples['Survived_spouse']==1 )]

    partial = couples.drop(survived.index)
    # return partial.groupby('Pclass')['Survived'].count()/couples.groupby('Pclass')['Survived'].count()
    return partial.groupby('Pclass').size()/couples.groupby('Pclass').size()


In [46]:
find_couples(df)

Pclass
1    0.780488
2    0.956522
3    1.000000
dtype: float64

In [30]:
PROBLEM_ID = 9

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, find_couples)

In [31]:
if TEST:
    print(f"{STUDENT}: {int(100 * total_grade / MAX_POINTS)}")